# <div style="font-family: 'Playfair Display', serif; font-weight:bold; letter-spacing: 0px; color:#FAF0EF; font-size:150%; text-align:center; padding:10px; background:#BEBF50; border-radius: 10px; box-shadow: 10px 10px 5px #ddad0fff;">🌟Homework 3 : CLASSIFICATION <br></div>

<a id="6"></a>
# <div style="font-family: 'Playfair Display', serif; font-weight:bold; letter-spacing: 1px; color:#FAF0EF; font-size:150%; text-align:left; padding:10px; background:#ddad0fff; border-radius: 10px;">📒 Question 1
</div>

<a id="6"></a>
# <div style="font-family: 'Playfair Display', serif; font-weight:bold; letter-spacing: 1px; color:#FAF0EF; font-size:150%; text-align:left; padding:10px; background:#ddad0fff; border-radius: 10px;">📒 Question 2
</div>

<a id="6"></a>
# <div style="font-family: 'Playfair Display', serif; font-weight:bold; letter-spacing: 1px; color:#FAF0EF; font-size:150%; text-align:left; padding:10px; background:#ddad0fff; border-radius: 10px;">📒 Question 3
</div>

<a id="6"></a>
# <div style="font-family: 'Playfair Display', serif; font-weight:bold; letter-spacing: 1px; color:#FAF0EF; font-size:150%; text-align:left; padding:10px; background:#ddad0fff; border-radius: 10px;">📒 Question 4
</div>

<a id="6"></a>
# <div style="font-family: 'Playfair Display', serif; font-weight:bold; letter-spacing: 1px; color:#FAF0EF; font-size:150%; text-align:left; padding:10px; background:#ddad0fff; border-radius: 10px;">📒 Question 5
</div>

<a id="6"></a>
# <div style="font-family: 'Playfair Display', serif; font-weight:bold; letter-spacing: 1px; color:#FAF0EF; font-size:150%; text-align:left; padding:10px; background:#ddad0fff; border-radius: 10px;"> 📒Question 6
</div>

<a id="6"></a>
# <div style="font-family: 'Playfair Display', serif; font-weight:bold; letter-spacing: 1px; color:#FAF0EF; font-size:150%; text-align:left; padding:10px; background:#ddad0fff; border-radius: 10px;">🔥 end by Amira mohammedi
</div>